# Kontrolowanie LimeSurveya za pomocą API

Bardzo wiele serwisów internetowych udostępnia API za pomocą którego można wchodzi w interakcje z danym serwisem bez używania do tego interfejsu graficznego, jedynie za pomocą skryptu. Również LimeSurvey (którego używaliśmy do robienia ankiet internetowych) udostępnia API. Dzięki różnym funkcjom z API będziemy w stanie wykonywać dużo powtarzalnych czynności (np. ściąganie danych badawczych, tworzenie długich ankiet).

Wszystkie funkcje dostępne za pomocą interfejsu RPC w LimeSurveyu dostępne są tutaj:

http://kognilab.pl/ls3/index.php/admin/remotecontrol

LimeSurvey oferuje dwa protokoły *remote procedure call* - jeden używający formatu JSON, drugi używający formatu XML. Z przyczyn historycznych na KogniLabowej instancji LimeSurveya używamy protokołu XML-RPC. W bibliotece standardowej Pythona znajduje się biblioteka do obsługi tego wystandaryzowanego protokołu - `xmlrpc`. Ma ona dwa moduły - kliencki i serwerowy. My nie chcemy pisać serwera obsługującego ten protokół, ale chcemy być klientem, dlatego użyjemy `xmlrpc.client`. 

Dodatkowo będą nam potrzebne dwie dodatkowe biblioteki `simplejson` (ten trzeba ściągnąć dodatkowo!) do pracy z formatem JSON (który LimeSurvey może zwracać) oraz `base64` do dekodowanie danych, które LimeSurvey koduje za pomocą algorytmu base64. 

Po zaimportowaniu modułów możemy autoryzować się na serwerze za pomocą loginu i hasła do LimeSurveya i otrzymać klucz sesji (*session key*), którym będziemy autoryzować się wykonując dalsze operacje.

In [108]:
from pprint import pprint, pformat # pprint i pformat to prettyprint i prettyformat
from base64 import b64decode
import simplejson as json
from xmlrpc.client import ServerProxy
# tworzymy obiekt reprezentujący interfejs RPC
kognilab = ServerProxy('http://kognilab.pl/ls3/index.php/admin/remotecontrol') 
login = 'Użytkownik Testowy'
password = 'test123'
# otrzymujemy klucz sesji
skey = kognilab.get_session_key(login, password)
skey

'7Elo8CtwyBawLCh~I7nOhrNSyk2yjOOy'

Teraz, kiedy jesteśmy już autoryzowani na serwerze możemy wyświetlić listę ankiet, do których mamy dostęp. Python automatycznie konwertuje zwracane dane do postaci słownika (`dict`):

In [95]:
surveys = kognilab.list_surveys(skey, # klucz sesji, to będzie zawsze PIERWSZY argument 
                                'Użytkownik Testowy') # chcemy tylko ankiety do których my mamy dostęp
pprint(surveys)

[{'active': 'Y',
  'expires': '',
  'sid': 537838,
  'startdate': '',
  'surveyls_title': 'Ankieta dydaktyczna'}]


Jeśli chcemy więcej dowiedzieć się o danej ankiecie, możemy skorzystać z funkcji `get_survey_properties`:

In [109]:
properties = kognilab.get_survey_properties(skey, # skey ZAWSZE pierwszy, jeśli coś nie działa to na 90% dlatego
                                            surveys[0]['sid']) # numer ankiety
pprint(properties)

{'active': 'Y',
 'additional_languages': '',
 'admin': 'Administrator',
 'adminemail': 'your-email@example.net',
 'alloweditaftercompletion': 'N',
 'allowprev': 'Y',
 'allowregister': 'N',
 'allowsave': 'Y',
 'anonymized': 'N',
 'assessments': 'N',
 'attributedescriptions': '',
 'autonumber_start': 0,
 'autoredirect': 'N',
 'bounce_email': 'your-email@example.net',
 'bounceaccountencryption': '',
 'bounceaccounthost': '',
 'bounceaccountpass': '',
 'bounceaccounttype': '',
 'bounceaccountuser': '',
 'bounceprocessing': 'N',
 'bouncetime': '',
 'datecreated': '2018-10-11 11:58:19',
 'datestamp': 'Y',
 'emailnotificationto': '',
 'emailresponseto': '',
 'expires': '',
 'faxto': '',
 'format': 'G',
 'googleanalyticsapikey': '',
 'googleanalyticsstyle': '',
 'gsid': 1,
 'htmlemail': 'Y',
 'ipaddr': 'Y',
 'language': 'pl',
 'listpublic': 'N',
 'navigationdelay': 0,
 'nokeyboard': 'N',
 'owner_id': 1,
 'printanswers': 'N',
 'publicgraphs': 'N',
 'publicstatistics': 'N',
 'questionindex': 2,


## Eksport wyników

Wyeksportować wyniki pozwala funkcja `export_responses`:

In [114]:
responses = kognilab.export_responses(skey, # session key zawsze pierwszy
                                      surveys[0]['sid'], # numer ankiety
                                      'json', # format, dostępne: 'csv', 'pdf', 'doc', 'json', 'xls'
                                      properties['language'], # język ankiety
                                     'all', # wszystkie odpowiedzi, dostępne: 'complete', 'incomplete', 'all'
                                     'full', # format nagłówków, dostępne 'code','full' lub 'abbreviated'
                                     'long') # format odpowiedzi, dostępne: 'short', 'long' 
responses[:600] # pierwsze 600 znaków żeby nie zaśmiecać notebooka...

'eyJyZXNwb25zZXMiOiBbeyIxIjp7IklEIG9kcG93aWVkemkiOiIxIiwiRGF0YSB3eXNcdTAxNDJhbmlhIjoiIiwiT3N0YXRuaWEgc3Ryb25hIjoiNiIsIkpcdTAxMTl6eWsgcG9jelx1MDEwNXRrb3d5IjoicGwiLCJOYXNpZW5pZSI6IjEwNzUxMDE3IiwiRGF0YSByb3pwb2N6XHUwMTE5Y2lhIjoiMjAxOC0xMC0xMiAxMDoyMzo0MiIsIkRhdGEgb3N0YXRuaWVqIGFrY2ppIjoiMjAxOC0xMC0xMiAxMDo1NTo1MCIsIkFkcmVzIElQIjoiMTkzLjAuMTA4LjQyIiwiT2RzeVx1MDE0MmFqXHUwMTA1Y3kgVVJMIjoiaHR0cDpcL1wva29nbmlsYWIucGxcL2xzM1wvaW5kZXgucGhwXC9hZG1pblwvc3VydmV5XC9zYVwvdmlld1wvc3VydmV5aWRcLzUzNzgzOCIsIlphIGNod2lsXHUwMTE5IHpvYmFjenlteSByb2R6YWplIHB5dGFcdTAxNDQsIGt0XHUwMGYzcmUgbW9cdTAxN2NuYSB3eWtvcnp5c3RhXHUw'

Problem jest taki, że zwracane są odpowiedzi zakodowane w formacie base64. Bez problemu możemy je odkodować używając funkcji `b64decode`.

In [115]:
decoded = b64decode(responses).decode() # .decode() konwertuje do stringa
decoded[:600] # pierwsze 600 znaków, żeby nie zaśmiecać notebooka

'{"responses": [{"1":{"ID odpowiedzi":"1","Data wys\\u0142ania":"","Ostatnia strona":"6","J\\u0119zyk pocz\\u0105tkowy":"pl","Nasienie":"10751017","Data rozpocz\\u0119cia":"2018-10-12 10:23:42","Data ostatniej akcji":"2018-10-12 10:55:50","Adres IP":"193.0.108.42","Odsy\\u0142aj\\u0105cy URL":"http:\\/\\/kognilab.pl\\/ls3\\/index.php\\/admin\\/survey\\/sa\\/view\\/surveyid\\/537838","Za chwil\\u0119 zobaczymy rodzaje pyta\\u0144, kt\\u00f3re mo\\u017cna wykorzysta\\u0107 z cz\\u0119\\u015bci badawczej ankiety.  To pytanie ma rodzaj \\"Text display\\". Ten rodzaj pytania pozwala dodawa\\u0107 do ankiety ekrany, na kt\\u00'

W wyniku tej operacji faktycznie otrzymaliśmy ciąg znaków w formacie JSON. Żeby go przekonwertować na pythonowski słownik (`dict`) użyjemy funkcji `loads` (*load string*) z biblioteki `simplejson`. 

In [116]:
as_dict = json.loads(decoded)
pprint(pformat(as_dict, indent=0)[:800]) # cuda na kiju żeby nie zaśmiecać

("{'responses': [{'1': {'Adres IP': '193.0.108.42',\n"
 "                  'Data ostatniej akcji': '2018-10-12 10:55:50',\n"
 "                  'Data rozpoczęcia': '2018-10-12 10:23:42',\n"
 "                  'Data wysłania': '',\n"
 "                  'ID odpowiedzi': '1',\n"
 "                  'Język początkowy': 'pl',\n"
 "                  'Nasienie': '10751017',\n"
 "                  'Odsyłający URL': "
 "'http://kognilab.pl/ls3/index.php/admin/survey/sa/view/surveyid/537838',\n"
 "                  'Ostatnia strona': '6',\n"
 "                  'Prosimy wybrać swoją płeć:  (To pytanie jest "
 'niepolityczne - dobrą praktyką jest pozostawienie opcji "Inne" oraz "Nie '
 'chcę udzielać odpowiedzi" - wtedy lepiej uzyć zwykłej listy)\': \'N/A\',\n'
 "                  'Pytania na skali Likerta najłatwiej zrobić za pomocą "
 'rodzaju pytania "Array". "Arr')


Zwykle nie będziemy chcieli, żeby dane były aż tak opisane. Spróbujmy zamiast pełnych nagłówków i pełnych odpowiedzi ściągnąć tylko ich kody:

In [117]:
responses = kognilab.export_responses(skey,
                                      surveys[0]['sid'],
                                      'json',
                                      properties['language'],
                                     'all',
                                     'code',
                                     'short')
as_dict = json.loads(b64decode(responses).decode())
pprint(pformat(as_dict, indent=0)[:800])

("{'responses': [{'1': {'G': None,\n"
 "                  'Q1': 'A2',\n"
 "                  'Q2': '',\n"
 "                  'Q3': 'A2',\n"
 "                  'Q4[SQ001]': 'A4',\n"
 "                  'Q5[SQ001]': '91',\n"
 "                  'Q5[SQ002]': '99',\n"
 "                  'Y': None,\n"
 "                  'badawczedisplay': '',\n"
 "                  'datestamp': '2018-10-12 10:55:50',\n"
 "                  'id': '1',\n"
 "                  'ipaddr': '193.0.108.42',\n"
 "                  'lastpage': '6',\n"
 "                  'refurl': "
 "'http://kognilab.pl/ls3/index.php/admin/survey/sa/view/surveyid/537838',\n"
 "                  'seed': '10751017',\n"
 "                  'startdate': '2018-10-12 10:23:42',\n"
 "                  'startlanguage': 'pl',\n"
 "                  'submitdate': None}},\n"
 "             {'2': {'G': None,\n"
 "                  'Q1': 'A2',\n"
 ' ')


Świetnie! Prawie jesteśmy w domu. Teraz musimy coś zrobić z osobliwym formatem w jakim dostaliśmy dane. Dostaliśmy je jako `dict`. Prosty *one-liner* przekonwertuje je do bardziej intuicyjnego formatu (lista, 1 element = 1 badany).

In [118]:
responses = [list(response.values())[0] for response in as_dict['responses']]
responses[1]

{'id': '2',
 'submitdate': None,
 'lastpage': '4',
 'startlanguage': 'pl',
 'seed': '1038795668',
 'startdate': '2018-10-12 10:25:20',
 'datestamp': '2018-10-12 10:37:53',
 'ipaddr': '193.0.101.210',
 'refurl': 'http://kognilab.pl/ls3/index.php/admin/survey/sa/view/surveyid/537838',
 'badawczedisplay': '',
 'Q1': 'A2',
 'Q2': 'A6',
 'Q3': 'A2',
 'Q4[SQ001]': None,
 'Q5[SQ001]': None,
 'Q5[SQ002]': None,
 'G': None,
 'Y': None}

Teraz możemy na przykład zobaczyć rok urodzenia uczestników badania (jeżeli nie podali w ankiecie wieku to oznaczamy to wartością 0)

In [119]:
lata_urodzenia = [int(response['Y']) if response['Y'] is not None else 0 for response in responses]
lata_urodzenia[:10]

[0, 0, 1996, 0, 0, 66666, 0, 0, 0, 0]

## Eksport ankiet do pliku CSV

Najbardziej przydatnym sposobem eksportowania ankiet za pomocą RPC jest eksport do plików CSV. Najłatwiej zrobić to w ten sposób:

In [106]:
responses = kognilab.export_responses(skey,
                                      surveys[0]['sid'], # numer ankiety
                                      'csv', # format pliku
                                      properties['language'], # język ankiety
                                     'completed', # status - "completed" albo "all"
                                     'code', # chcemy tylko kody odpowiedzi
                                     'short') # krótkie nazwy kolumn
csv = b64decode(responses).decode()
try:
    with open(str(surveys[0]['sid'])+'.csv', 'w') as f:
        f.write(csv)
    print('Udało się zapisać plik!')
except Exception as e:
    print('Coś poszło nie tak...')
    print(e)

Udało się zapisać plik!
